# 1. Combinar archivos 
#### Archivo de enrgia extraida con archivo de empresas

In [1]:
import pandas as pd
import glob
import re
import os
from pathlib import Path

# Configuración de paths
# Configuración
EMPRESAS_FILE = './data/1_empresas.xlsx'
INPUT_PATTERN = './preprocess/extracted_ingresos_c_ret_*.xlsx'
OUTPUT_PREFIX = './preprocess/ingresos_empresas_'

def procesar_archivos():
    # Cargar el mapeo de empresas
    try:
        df_empresas = pd.read_excel(EMPRESAS_FILE)
        
        # Verificar columnas requeridas
        if 'AGENTE' not in df_empresas.columns or 'EMPRESA' not in df_empresas.columns:
            print("Error: El archivo empresas.xlsx debe contener columnas 'AGENTE' y 'EMPRESA'")
            return
            
        # Crear diccionarios para mapeo
        df_empresas['AGENTE'] = df_empresas['AGENTE'].astype(str).str.strip()
        mapeo_empresas = dict(zip(df_empresas['AGENTE'], df_empresas['EMPRESA']))
        nombres_empresas = {k: k for k in df_empresas['AGENTE'].unique()}
        
    except Exception as e:
        print(f"Error cargando {EMPRESAS_FILE}: {str(e)}")
        return

    # Procesar cada archivo de entrada
    for input_file in glob.glob(INPUT_PATTERN):
        file_number = re.search(r'extracted_ingresos_c_ret_(\d+)\.xlsx', input_file)
        if not file_number:
            continue
        
        file_number = file_number.group(1)
        output_file = f"{OUTPUT_PREFIX}{file_number}.xlsx"
        
        if os.path.exists(output_file):
            print(f"Archivo {output_file} ya existe. Omitiendo.")
            continue

        try:
            # Leer datos de energía
            df_energia = pd.read_excel(input_file)
            
            # Limpieza inicial de columnas
            df_energia.columns = [col.strip() for col in df_energia.columns]
            
            # Renombrar columna de agente si es necesario
            agent_col = next((col for col in df_energia.columns if 'agente' in col.lower()), None)
            if agent_col and agent_col != 'AGENTE':
                df_energia = df_energia.rename(columns={agent_col: 'AGENTE'})
            
            if 'AGENTE' not in df_energia.columns:
                print(f"Error: No se encontró columna 'AGENTE' en {input_file}")
                continue

            # Convertir AGENTE a string y limpiar
            df_energia['AGENTE'] = df_energia['AGENTE'].astype(str).str.strip()
            
            # Función para normalizar nombres
            def normalizar_nombre(x):
                x = str(x).strip()
                # Primero buscar coincidencia exacta
                if x in nombres_empresas:
                    return nombres_empresas[x]
                # Luego buscar sin guiones/espacios
                x_clean = x.replace('-', '').replace(' ', '')
                for k in nombres_empresas:
                    if k.replace('-', '').replace(' ', '') == x_clean:
                        return nombres_empresas[k]
                return x
            
            # Asignar empresa y nombre normalizado
            df_energia['AGENTE_NORMALIZADO'] = df_energia['AGENTE'].apply(normalizar_nombre)
            df_energia['EMPRESA'] = df_energia['AGENTE_NORMALIZADO'].map(mapeo_empresas)

            # Reordenar columnas
            cols = ['AGENTE_NORMALIZADO', 'EMPRESA'] + [
                col for col in df_energia.columns 
                if col not in ['AGENTE_NORMALIZADO', 'EMPRESA', 'AGENTE']
            ]
            
            # Renombrar y guardar
            df_resultado = df_energia[cols].rename(columns={'AGENTE_NORMALIZADO': 'AGENTE'})
            df_resultado.to_excel(output_file, index=False)
            print(f"Procesado: {input_file} -> {output_file}")

            # Reportar inconsistencias
            if df_resultado['EMPRESA'].isna().any():
                missing = df_resultado[df_resultado['EMPRESA'].isna()]['AGENTE'].unique()
                print(f"  Advertencia: {len(missing)} agentes sin empresa asignada")
                if len(missing) > 3:
                    print(f"  Ejemplos: {missing[:3]}...")
                else:
                    print(f"  {missing}")

        except Exception as e:
            print(f"Error procesando {input_file}: {str(e)}")
            # Guardar archivo con error para diagnóstico
            error_file = os.path.join(OUTPUT_PREFIX, f"ERROR_{os.path.basename(output_file)}")
            df_energia.to_excel(error_file, index=False)
            print(f"  Se guardó copia del archivo problemático como {error_file}")

if __name__ == "__main__":
    print("Iniciando procesamiento...")
    procesar_archivos()
    print("Proceso completado.")

Iniciando procesamiento...
Procesado: ./preprocess\extracted_ingresos_c_ret_0123.xlsx -> ./preprocess/ingresos_empresas_0123.xlsx
  Advertencia: 8 agentes sin empresa asignada
  Ejemplos: ['nan' 'CARGOS POR RETIROS DE ENERGIA Y POTENCIA (sin IVA)'
 '2023-01-01 00:00:00']...
Procesado: ./preprocess\extracted_ingresos_c_ret_0124.xlsx -> ./preprocess/ingresos_empresas_0124.xlsx
  Advertencia: 17 agentes sin empresa asignada
  Ejemplos: ['nan' 'CARGOS POR RETIROS DE ENERGIA Y POTENCIA (sin IVA)'
 'enero de 2024']...
Procesado: ./preprocess\extracted_ingresos_c_ret_0125.xlsx -> ./preprocess/ingresos_empresas_0125.xlsx
  Advertencia: 13 agentes sin empresa asignada
  Ejemplos: ['nan' 'CARGOS POR RETIROS DE ENERGÍA Y POTENCIA (sin IVA)'
 'enero de 2025']...
Procesado: ./preprocess\extracted_ingresos_c_ret_0223.xlsx -> ./preprocess/ingresos_empresas_0223.xlsx
  Advertencia: 8 agentes sin empresa asignada
  Ejemplos: ['nan' 'CARGOS POR RETIROS DE ENERGIA Y POTENCIA (sin IVA)'
 '2023-02-01 00:00

# 2. Merge archivos energia

In [7]:
import pandas as pd
import glob
from datetime import datetime

def consolidar_en_formato_largo():
    archivos = sorted(glob.glob('./preprocess/ingresos_empresas_*.xlsx'))
    if not archivos:
        print("No se encontraron archivos.")
        return

    registros = []

    for archivo in archivos:
        try:
            periodo = archivo.split('_')[-1].split('.')[0]
            mes = int(periodo[:2])
            año = 2000 + int(periodo[2:])
            fecha = datetime(año, mes, 1)

            df = pd.read_excel(archivo)
            if 'AGENTE' not in df.columns:
                print(f"Omitido: {archivo} no tiene columna AGENTE.")
                continue

            for col in df.columns:
                if col in ['AGENTE', 'EMPRESA']:
                    continue
                temp = df[['AGENTE', 'EMPRESA', col]].copy()
                temp['FECHA'] = fecha
                temp['VARIABLE'] = col
                temp = temp.rename(columns={col: 'VALOR'})
                registros.append(temp)

        except Exception as e:
            print(f"Error procesando {archivo}: {e}")
            continue

    if not registros:
        print("No se pudo consolidar ningún archivo válido.")
        return

    df_largo = pd.concat(registros, ignore_index=True)
    df_largo = df_largo[['FECHA', 'AGENTE', 'EMPRESA', 'VARIABLE', 'VALOR']]

    df_largo.to_excel("./preprocess/serie_temporal_ingresos.xlsx", index=False)
    print("Consolidación completada en formato largo.")
    print(f"Filas totales: {len(df_largo)}")
    print(f"Archivo guardado como 'serie_temporal_ingresos.xlsx'")

if __name__ == "__main__":
    consolidar_en_formato_largo()

Consolidación completada en formato largo.
Filas totales: 6294
Archivo guardado como 'serie_temporal_ingresos.xlsx'


# 3. Creacion de serie de tiempo de energia

In [3]:
import pandas as pd

# Leer archivo
df = pd.read_excel("./preprocess/serie_temporal_ingresos.xlsx")

# Convertir FECHA y eliminar filas sin fecha válida
df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce', unit='d')
df = df.dropna(subset=['FECHA'])

# Limpiar espacios en columnas clave
df['AGENTE'] = df['AGENTE'].astype(str).str.strip()
df['EMPRESA'] = df['EMPRESA'].astype(str).str.strip()
df['VARIABLE'] = df['VARIABLE'].astype(str).str.strip()

# Filtrar sólo las variables de interés exactas (con mayúsculas y espacios)
df = df[df['VARIABLE'].isin([
                        "Energía MWh",
                        'Ingresos Energía MWh',
                        'Ingresos Renovables MWh',
                        'Ingresos Potencia kW'
                    ])]

# Asegurar que VALOR sea numérico
df['VALOR'] = pd.to_numeric(df['VALOR'], errors='coerce')
df = df.dropna(subset=['VALOR'])

# Crear MES_ANIO
df['MES_ANIO'] = df['FECHA'].dt.strftime('%m%Y')

# Reemplazar valores faltantes en AGENTE y EMPRESA con forward fill
df['AGENTE'] = df['AGENTE'].replace('nan', pd.NA).fillna(method='ffill')
df['EMPRESA'] = df['EMPRESA'].replace('nan', pd.NA).fillna(method='ffill')

# Crear columna pivote combinando variable y mes_año (con espacios y mayúsculas)
df['COLUMNA'] = df['VARIABLE'] + ' ' + df['MES_ANIO']

# Pivotear el DataFrame
df_pivot = df.pivot_table(index=['AGENTE', 'EMPRESA'], columns='COLUMNA', values='VALOR', aggfunc='first')

# Función para extraer año y mes para ordenar correctamente
def extraer_anio_mes(col_name):
    fecha = col_name.split()[-1]  # '012023'
    mes = int(fecha[:2])
    anio = int(fecha[2:])
    return (anio, mes)

# Obtener lista de columnas
cols = df_pivot.columns.tolist()

# Separar en Energía y Potencia
energia_cols = sorted([c for c in cols if c.startswith('Energía MWh')], key=extraer_anio_mes)
ingresos_energia_cols = sorted([c for c in cols if c.startswith('Ingresos Energía MWh')], key=extraer_anio_mes)
ingresos_renovables_cols = sorted([c for c in cols if c.startswith('Ingresos Renovables MWh')], key=extraer_anio_mes)  
ingresos_potencia_cols = sorted([c for c in cols if c.startswith('Ingresos Potencia kW')], key=extraer_anio_mes)  

  
# Concatenar para orden final
cols_ordenadas = energia_cols + ingresos_energia_cols + ingresos_renovables_cols + ingresos_potencia_cols

# Reordenar columnas
df_pivot = df_pivot[cols_ordenadas]

# Resetear índice
df_final = df_pivot.reset_index()

# Guardar resultado
df_final.to_excel("./preprocess/serie_ingresos_cronologica.xlsx", index=False)

C:\Users\daniel.canedo\AppData\Local\Temp\ipykernel_34204\3119654209.py:31: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['AGENTE'] = df['AGENTE'].replace('nan', pd.NA).fillna(method='ffill')
C:\Users\daniel.canedo\AppData\Local\Temp\ipykernel_34204\3119654209.py:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['EMPRESA'] = df['EMPRESA'].replace('nan', pd.NA).fillna(method='ffill')


# 4. Depuracion de serie de tiempo de energia

In [4]:
import pandas as pd
import numpy as np

# Leer el archivo Excel
df = pd.read_excel("./preprocess/serie_ingresos_cronologica.xlsx")

# Lista de agentes a eliminar
agentes_a_eliminar = [
    "TOTAL - CESSA", "TOTAL - CRE R.L.", "TOTAL CRE", "TOTAL - DELAPAZ", 
    "TOTAL - ELFEC", "TOTAL - ENDE", "TOTAL ENDE DELBENI S.A.M.",
    "TOTAL - ENDE DEORURO S.A.", "TOTALES", "Tipo de cambio",
    "TOTAL - SEPSA", "TOTAL - SETAR"
]

# Filtrar y eliminar las filas no deseadas
df = df[~df["AGENTE"].isin(agentes_a_eliminar)]

# Convertir todas las columnas numéricas (remover comas y convertir a float)
numeric_cols = df.columns.drop(['AGENTE', 'EMPRESA'])
for col in numeric_cols:
    if df[col].dtype == object:
        df[col] = df[col].str.replace(',', '', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Identificar periodos únicos (ej: 012023, 022023, etc)
periods = []
for col in df.columns:
    if col.startswith('Energía MWh '):
        period = col.split(' ')[-1]
        periods.append(period)

# Calcular precio monómico para cada periodo con el nombre de columna corregido
for period in periods:
    energia_col = f'Energía MWh {period}'
    ing_ener_col = f'Ingresos Energía MWh {period}'
    ing_ren_col = f'Ingresos Renovables MWh {period}'
    ing_pot_col = f'Ingresos Potencia kW {period}'
    precio_col = f'Precio Monómico USD/MWh {period}'  # Nombre corregido
    
    # Verificar que existen las columnas necesarias
    if all(col in df.columns for col in [energia_col, ing_ener_col, ing_ren_col, ing_pot_col]):
        # Calcular suma de ingresos
        total_ingresos = df[ing_ener_col] + df[ing_ren_col] + df[ing_pot_col]
        
        # Calcular precio monómico evitando división por cero
        df[precio_col] = np.where(
            df[energia_col] > 0,
            (total_ingresos / df[energia_col]) * 1000,  # USD/MWh
            np.nan
        )
    else:
        print(f"Advertencia: Columnas incompletas para el período {period}")

# Guardar el DataFrame con las nuevas columnas
df.to_excel("./data/serie_ingresos.xlsx", index=False)

print("Proceso completado: Se han añadido las columnas de precio monómico con unidades USD/MWh")

Proceso completado: Se han añadido las columnas de precio monómico con unidades USD/MWh
